# Chunked Pooling
This notebooks explains how the chunked pooling can be implemented. First you need to install the requirements: 

In [1]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


Then we load a model which we want to use for the embedding. We choose `jinaai/jina-embeddings-v2-base-en` but any other model which supports mean pooling is possible. However, models with a large maximum context-length are preferred.

In [2]:
from chunked_pooling import chunked_pooling, chunk_by_sentences

In [3]:
from transformers import AutoModel
from transformers import AutoTokenizer

# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('jinaai/jina-embeddings-v3', trust_remote_code=True)
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v3', trust_remote_code=True)

Now we define the text which we want to encode and split it into chunks. The `chunk_by_sentences` function also returns the span annotations. Those specify the number of tokens per chunk which is needed for the chunked pooling.

In [4]:
input_text = "Москва — столица России, город федерального значения, административный центр Центрального федерального округа и центр Московской области, в состав которой не входит. Мегаполис; крупнейший по численности населения город России и её субъект — 13 149 803 человека (2024), что делает Москву 22-й среди городов мира по численности населения. Центр Московской городской агломерации. Самый крупный город Европы по площади и населению."
#input_text = "Berlin is the capital and largest city of Germany, both by area and by population. Its more than 3.85 million inhabitants make it the European Union's most populous city, as measured by population within city limits. The city is also one of the states of Germany, and is the third smallest state in the country in terms of area."

# determine chunks
chunks, span_annotations = chunk_by_sentences(input_text, tokenizer)
print('Chunks:\n- "' + '"\n- "'.join(chunks) + '"')


Chunks:
- "Москва — столица России, город федерального значения, административный центр Центрального федерального округа и центр Московской области, в состав которой не входит."
- " Мегаполис; крупнейший по численности населения город России и её субъект — 13 149 803 человека (2024), что делает Москву 22-й среди городов мира по численности населения."
- " Центр Московской городской агломерации."
- " Самый крупный город Европы по площади и населению."


Now we encode the chunks with the traditional and the context-sensitive chunked pooling method:

In [5]:
# chunk before
embeddings_traditional_chunking = model.encode(chunks)

# chunk afterwards (context-sensitive chunked pooling)
inputs = tokenizer(input_text, return_tensors='pt')
model_output = model(**inputs)
embeddings = chunked_pooling(model_output, [span_annotations])[0]

Finally, we compare the similarity of the word "Berlin" with the chunks. The similarity should be higher for the context-sensitive chunked pooling method:

In [6]:
import numpy as np

cos_sim = lambda x, y: np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

test_text = 'Москва'#"Berlin"#
berlin_embedding = model.encode(test_text)

for chunk, new_embedding, trad_embeddings in zip(chunks, embeddings, embeddings_traditional_chunking):
    print(f'similarity_new("{test_text}", "{chunk}"):', cos_sim(berlin_embedding, new_embedding))
    print(f'similarity_trad("{test_text}", "{chunk}"):', cos_sim(berlin_embedding, trad_embeddings))

similarity_new("Москва", "Москва — столица России, город федерального значения, административный центр Центрального федерального округа и центр Московской области, в состав которой не входит."): 0.72507524
similarity_trad("Москва", "Москва — столица России, город федерального значения, административный центр Центрального федерального округа и центр Московской области, в состав которой не входит."): 0.78409195
similarity_new("Москва", " Мегаполис; крупнейший по численности населения город России и её субъект — 13 149 803 человека (2024), что делает Москву 22-й среди городов мира по численности населения."): 0.7113851
similarity_trad("Москва", " Мегаполис; крупнейший по численности населения город России и её субъект — 13 149 803 человека (2024), что делает Москву 22-й среди городов мира по численности населения."): 0.6820705
similarity_new("Москва", " Центр Московской городской агломерации."): 0.7089583
similarity_trad("Москва", " Центр Московской городской агломерации."): 0.6953759
sim

In [7]:
for chunk, new_embedding, trad_embeddings in zip(chunks, embeddings, embeddings_traditional_chunking):
    print(f'abs new("{chunk}"):', np.linalg.norm(new_embedding))
    print(f'abs trad("{chunk}"):', np.linalg.norm(trad_embeddings))

print(f'\nabs test_text("{test_text}"):', np.linalg.norm(berlin_embedding))

abs new("Москва — столица России, город федерального значения, административный центр Центрального федерального округа и центр Московской области, в состав которой не входит."): 19.243435
abs trad("Москва — столица России, город федерального значения, административный центр Центрального федерального округа и центр Московской области, в состав которой не входит."): 1.0
abs new(" Мегаполис; крупнейший по численности населения город России и её субъект — 13 149 803 человека (2024), что делает Москву 22-й среди городов мира по численности населения."): 18.395842
abs trad(" Мегаполис; крупнейший по численности населения город России и её субъект — 13 149 803 человека (2024), что делает Москву 22-й среди городов мира по численности населения."): 1.0
abs new(" Центр Московской городской агломерации."): 17.773409
abs trad(" Центр Московской городской агломерации."): 1.0
abs new(" Самый крупный город Европы по площади и населению."): 19.818975
abs trad(" Самый крупный город Европы по площади и 

In [8]:
for chunk, new_embedding, trad_embeddings in zip(chunks, embeddings, embeddings_traditional_chunking):
    print(f'std new("{chunk}"):', np.std(new_embedding))
    print(f'std trad("{chunk}"):', np.std(trad_embeddings))

print(f'\nstd test_text("{test_text}"):', np.std(berlin_embedding))

std new("Москва — столица России, город федерального значения, административный центр Центрального федерального округа и центр Московской области, в состав которой не входит."): 0.6013478
std trad("Москва — столица России, город федерального значения, административный центр Центрального федерального округа и центр Московской области, в состав которой не входит."): 0.031249119
std new(" Мегаполис; крупнейший по численности населения город России и её субъект — 13 149 803 человека (2024), что делает Москву 22-й среди городов мира по численности населения."): 0.5748528
std trad(" Мегаполис; крупнейший по численности населения город России и её субъект — 13 149 803 человека (2024), что делает Москву 22-й среди городов мира по численности населения."): 0.031249631
std new(" Центр Московской городской агломерации."): 0.55538654
std trad(" Центр Московской городской агломерации."): 0.031248007
std new(" Самый крупный город Европы по площади и населению."): 0.6193383
std trad(" Самый крупный г